In [1]:
from scholarly import scholarly
import pandas as pd

In [2]:
author = list(scholarly.search_author("Taylor Salo"))[0]
author = scholarly.fill(author)
pubs = author["publications"]

In [3]:
def convert_names(names):
    # Split the string into a list of names
    names = names.split(" and ")

    # Initialize an empty list to hold the converted names
    converted_names = []

    # Loop over each name in the list
    for name in names:
        # Split the name into first and last names
        name_parts = name.split()
        last = name_parts[-1]
        first = " ".join(name_parts[:-1])

        # Convert the first name into initials
        initials = ''.join([i[0] for i in first.split()])
        
        converted_name = f"{last} {initials}"
        if converted_name == "Salo T":
            converted_name = "\\textbf{Salo T}"

        # Format the last name and initials and add them to the list
        converted_names.append(converted_name)

    # Join the converted names into a string with commas and return it
    out_str = ", ".join(converted_names[:-1])
    out_str += f", \& {converted_names[-1]}"
    return out_str

In [4]:
sorted_pubs = sorted(pubs, key=lambda d: int(d["bib"].get("pub_year", 1990)))[::-1]

In [9]:
for pub in sorted_pubs:
    pub_info = scholarly.fill(pub)
    bib = pub_info["bib"]
    name_str = convert_names(bib["author"])
    if 'journal' in bib.keys():
        journal = bib["journal"]
    else:
        journal = bib["citation"].split(",")[0]

    citation = f"\\item {name_str}.\n({bib['pub_year']}).\n{bib['title']}.\n\\emph{{{journal}}}.\n"
    print(citation)

\item Zhao C, Jarecka D, Covitz S, Chen Y, Eickhoff SB, Fair DA, Franco AR, Halchenko YO, Hendrickson TJ, Hoffstaedter F, Houghton A, Kiar G, Macdonald A, Mehta K, Milham MP, \textbf{Salo T}, Hanke M, Ghosh SS, Cieslak M, \& Satterthwaite TD. (2024)
A reproducible and generalizable software workflow for analysis of large-scale neuroimaging data collections using BIDS Apps
\emph{Imaging Neuroscience}.

\item Peraza JA, \textbf{Salo T}, Riedel MC, Bottenhorn KL, Poline J, Dockès J, Kent JD, Bartley JE, Flannery JS, Hill-Bowen LD, Lobo RP, Poudel R, Ray KL, Robinson JL, Laird RW, Sutherland MT, Vega Adl, \& Laird AR. (2024)
Methods for decoding cortical gradients of functional connectivity
\emph{Imaging Neuroscience}.

\item Peraza JA, \textbf{Salo T}, Riedel MC, Bottenhorn KL, Poline J, Dockès J, Kent JD, Bartley JE, Flannery JS, Hill-Bowen LD, Lobo RP, Poudel R, Ray KL, Robinson JL, Laird RW, Sutherland MT, Vega Adl, \& Laird AR. (2023)
Meta-analytic decoding of the cortical gradient of

KeyError: 'pub_year'

In [8]:
bib["citation"]

'Trends in Neuroscience and Education, 100204, 2023'